In [12]:
import csv
import os
from datetime import datetime
import json
import psycopg2
import sys
sys.path.insert(0,'..')

from scripts.database import get_conn_pool, execute, fetch_all
from scripts.routing import route_nogo, route_standard

In [13]:
# connect to postgis database
pool = get_conn_pool({
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'postgres',
    'database': 'xlab-routing'
})

In [14]:
# set start and end points

# START COORDINATES
start_lat = -1.271346
start_long = 36.751293

# END COORDINATES
end_lat = -1.190509
end_long = 36.726833

In [4]:
# set nogo layers
nogo_layer_list = ['hgis_admin3', 'hgis_bridges_tunnels']

# set nogo layer queries
nogo_layer_query_list = ["hgis_admin3.adm3_name = 'Kitisuru'", 
                         "hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811"]

# set nogo point layer buffer distance (in degrees) 
nogo_point_buffer = .0005  #50 meters

# set nogo polyline layer buffer distance (in degrees)
nogo_line_buffer = .0001  #10 meters


In [15]:
# get route that avoids nogo geometry
route = route_nogo(nogo_layer_list, start_long, start_lat, end_long, end_lat, nogo_layer_query_list)
wkt = route[0]
length = route[1]

# get standard route
# route2 = route_standard(start_long, start_lat, end_long, end_lat)
# wkt2 = route2[0]
# length2 = route2[1]
# wkt


NOGO QUERY:  SELECT ST_GEOMFROMEWKT(ST_COLLECT(a.shape)) FROM (SELECT ST_FORCE2D(shape) as shape FROM hgis_admin3 WHERE hgis_admin3.adm3_name = 'Kitisuru' UNION SELECT ST_FORCE2D(ST_BUFFER(shape, 0.0005)) as shape FROM hgis_bridges_tunnels WHERE hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811) as a
START NODE:  10339
END NODE:  57155


In [16]:
from ipyleaflet import Map, WKTLayer
import re

m = Map(center=[start_lat, end_long], zoom=12)
wlayer = WKTLayer(
    wkt_string=wkt,
    hover_style={"fillColor": "red"},
)
m.add_layer(wlayer)

# wlayer2 = WKTLayer(
#     wkt_string=wkt2,
#     hover_style={"fillColor": "green"},
# )
# m.add_layer(wlayer2)

m

Map(center=[-1.271346, 36.726833], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…